<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Sequential_Features_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Function

In [2]:
!pip install imbalanced-learn
!pip install ipywidgets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from joblib import Parallel, delayed
from google.colab import files
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)


def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)


def save_model(model,model_name):
 joblib.dump(model, model_name)
  

def load_model(model_name):
  return joblib.load(model_name)


def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  return dic



def divide_dataset(dataset,percentage,random_state = 0):
  rows,cols = dataset.shape
  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]



def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)



def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)


def tune_report_csv(model,dic,feat_selection_method,number_of_feat):

  tunning_type = []
  accuracy = []
  scaled_accuracy = []
  model_name = []
  feature_selection_method = []
  number_of_features = []

  li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]

  for i in list(dic.keys()):
    X_train, X_test, y_train, y_test = dic[i]
    
    if i in li and str(model[:3]) == "xgb":
      model.fit(X_train,y_train.values)
      y_pred = model.predict(X_test.values)
    else:
      model.fit(X_train,y_train)
      y_pred = model.predict(X_test)

    accu =accuracy_score(y_test, y_pred)
    scaled_accu =  float("{:.2f}".format(accu))
    print(i)
    print("Accurecy: ",scaled_accu)
    tunning_type.append(i)
    accuracy.append(accu)
    scaled_accuracy.append(scaled_accu)
    model_name.append(model)
    feature_selection_method.append(feat_selection_method)
    number_of_features.append(number_of_feat)
    
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")

  csv_dic={
      "model_name"  : model_name,
      "tunning_type" : tunning_type,
      "accurecy"    : accuracy, 
      "scaled_accuracy" :scaled_accuracy,
      "feature_selection_method" : feature_selection_method,
      "number_of_features" : number_of_features
  }


  df = pd.DataFrame.from_dict(csv_dic)
  df.to_csv("/content/drive/MyDrive/Sleep Stage 5 class/Features_selection_result/"+feat_selection_method+"_"+str(number_of_feat)+"_"+str(model)+".csv",index = False)
  return "Tunning Data Report is Secured for this phase!!!"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.3 MB/s eta 0:00:00


#Starting

In [3]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

#dataset = divide_dataset(dataset,0.04611,1)
#print(dataset.shape)

(108920, 52)
(108450, 52)


###Spliting into X and y

In [4]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108450 entries, 0 to 108449
Data columns (total 49 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   MeanP_Alpha_F4           108450 non-null  float64
 1   MeanP_Beta_F4            108450 non-null  float64
 2   MeanP_Theta_F4           108450 non-null  float64
 3   MeanP_Delta_F4           108450 non-null  float64
 4   MeanP_Gamma_F4           108450 non-null  float64
 5   MeanP_Alpha_C4           108450 non-null  float64
 6   MeanP_Beta_C4            108450 non-null  float64
 7   MeanP_Theta_C4           108450 non-null  float64
 8   MeanP_Delta_C4           108450 non-null  float64
 9   MeanP_Gamma_C4           108450 non-null  float64
 10  MeanP_Alpha_O2           108450 non-null  float64
 11  MeanP_Beta_O2            108450 non-null  float64
 12  MeanP_Theta_O2           108450 non-null  float64
 13  MeanP_Delta_O2           108450 non-null  float64
 14  Mean

#Feature Selection

In [ ]:
n = [15,40]

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

for number_of_feat in n:
  sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
  sfs.fit(X, y)
  list_of_feat=[]
  list_of_feat=list(sfs.get_feature_names_out(X.columns))

  df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

  df.to_csv("/content/drive/MyDrive/Sleep Stage 5 class/Sequential Featres/"+number_of_feat+"_Filter_Method_Sequential_feat_Selection_KNN.csv")

